# **Fine-tuning Wav2Vec2 for English ASR with 🤗 Transformers**

### Fine-tuning with Korean English accent dataset 

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.

Using a novel contrastive pretraining objective, Wav2Vec2 learns powerful speech representations from more than 50.000 hours of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/wav2vec2.png)

For the first time, it has been shown that pretraining, followed by fine-tuning on very little labeled speech data achieves competitive results to state-of-the-art ASR systems. Using as little as 10 minutes of labeled data, Wav2Vec2 yields a word error rate (WER) of less than 5% on the clean test set of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) - *cf.* with Table 9 of the [paper](https://arxiv.org/pdf/2006.11477.pdf).

In this notebook, we will give an in-detail explanation of how Wav2Vec2's pretrained checkpoints can be fine-tuned on any English ASR dataset. Note that in this notebook, we will fine-tune Wav2Vec2 without making use of a language model. It is much simpler to use Wav2Vec2 without a language model as an end-to-end ASR system and it has been shown that a standalone Wav2Vec2 acoustic model achieves impressive results. For demonstration purposes, we fine-tune the "base"-sized [pretrained checkpoint](https://huggingface.co/facebook/wav2vec2-base) on the rather small [Timit](https://huggingface.co/datasets/timit_asr) dataset that contains just 5h of training data.

Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `soundfile` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [1]:
%%capture
#!pip install datasets==1.8.0
#!pip install git+https://github.com/huggingface/datasets # new dataset
!pip install transformers==4.4.0
!pip install soundfile
!pip install jiwer




---

${}^1$ Timit is usually evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. 

In 🤗 Transformers, the Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Timit and define a vocabulary based on the dataset's transcriptions.

Let's start by loading the dataset and taking a look at its structure.

## Korean - English dataset

 This Korean-english speech dataset is from https://aiopen.etri.re.kr/(Language only supports Korean), open data API service by ETRI(Electronics and Telecommunications Research Institute) from Korea. This datase contains pcm recording files of each 50 Korean native people reading 100 english sentences, and its matching text.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob
import random
# All files ending with .txt and pcm
text_files = glob.glob("/content/drive/MyDrive/korean_english/files/*/*.txt")
pcm_files = glob.glob("/content/drive/MyDrive/korean_english/files/*/*.pcm") 

print('text file length : ', len(text_files))
print('text file length : ', len(pcm_files))
print(random.sample(text_files,5))
print(random.sample(pcm_files,5))

text file length :  4887
text file length :  4888
['/content/drive/MyDrive/korean_english/files/I4F02224SDY0/SW201105ETRNI4F02224SDY0021.txt', '/content/drive/MyDrive/korean_english/files/GSM04233KMS0/SW201105ETRNGSM04233KMS0088.txt', '/content/drive/MyDrive/korean_english/files/GSM03224CHD0/SW201105ETRNGSM03224CHD0020.txt', '/content/drive/MyDrive/korean_english/files/I4F05625JSY0/SW201105ETRNI4F05625JSY0060.txt', '/content/drive/MyDrive/korean_english/files/I4F02821YRR0/SW201105ETRNI4F02821YRR0070.txt']
['/content/drive/MyDrive/korean_english/files/GSF09844KKS0/SW201105ETRNGSF09844KKS0038.pcm', '/content/drive/MyDrive/korean_english/files/I4F07443LJA0/SW201105ETRNI4F07443LJA0040.pcm', '/content/drive/MyDrive/korean_english/files/GSF05432KSY0/SW201105ETRNGSF05432KSY0056.pcm', '/content/drive/MyDrive/korean_english/files/GSF08233NEH0/SW201105ETRNGSF08233NEH0049.pcm', '/content/drive/MyDrive/korean_english/files/I4F02821YRR0/SW201105ETRNI4F02821YRR0065.pcm']


### Korean English

First, read text files, drop files with errors and also save following filenames 

In [ ]:
texts = []
filenames = []
for t in text_files:
    f = open(t, "r")
    try:
        temp = f.read()
    except:
        continue
    texts.append(temp.strip())
    filenames.append(t)

In [ ]:
print(texts[:5])

['He was standing with his mouth open staring at me', "We've had a lot of complaints from customers", "What's the url for your personal website I'd love to see it", "Yeah I don't know what's up with me I just drop straight to sleep", 'I was hoping to have a brief word with you']


In [ ]:
# Save text.txt and filename to directory

with open("/content/drive/MyDrive/korean_english/text.txt", "w") as f:
    for item in texts:
        f.write("%s\n" % item)

with open("/content/drive/MyDrive/korean_english/filename.txt", "w") as f:
    for item in filenames:
        f.write("%s\n" % item)

In [7]:
# Read text.txt and filename from directory

f = open("/content/drive/MyDrive/korean_english/text.txt", "r")
texts = f.read().splitlines()

f = open("/content/drive/MyDrive/korean_english/filename.txt", "r")
filenames = f.read().splitlines()

In [8]:
# checking alignment of texts and filenames

assert len(texts) == len(filenames), "texts and filenames are not matched perfectly"

For not romanized word like numbers and abbreviations are noted as a format like '[2]/[two]', so we will change those formats to second choice(eg. [2]/[two] will be processed to 'two')

In [9]:
print(texts[43])

You have no idea how excited I am for iron man [2]/[two]


In [10]:
# Preprocessing phrases like '[2]/[two]' to 'two'

new_texts = []
for i,text in enumerate(texts):
    if texts[i].find('/') == -1:
        new_texts.append(texts[i])
        continue
    temp_1 = texts[i]

    while(temp_1.find('/') != -1):
        index_slash = temp_1.find('/')
        index_start = temp_1[:index_slash].find('[')
        index_end = temp_1[index_slash:].find(']')
        target = temp_1[index_start:index_slash + 1 + index_end]
        temp = temp_1[index_start:index_slash + 1 + index_end].split('/')
        temp[1][1:-1]
        temp_1 = temp_1.replace(target,temp[1][1:-1])
    new_texts.append(temp_1)

In [11]:
print(new_texts[43])

You have no idea how excited I am for iron man two


Alright! The transcriptions look very clean and the language seems to correspond more to written text than dialogue. This makes sense taking into account that [Timit](https://huggingface.co/datasets/timit_asr) is a read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [12]:
import re

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

for i in range(len(new_texts)):
    new_texts[i] = re.sub(chars_to_ignore_regex, '', new_texts[i]).lower() + " "

In [ ]:
random.sample(new_texts,10)

['what should i do if my baggage is over the limit ',
 'can i get a ticket for the sightseeing bus ',
 'what kind of newspapers would you like ',
 "you can play computer games but not this late you came home at ten o'clock ",
 'to say or shout something loudly because you want someone to hear you ',
 'is it possible to tee off now that we have a foursome ',
 'i wonder what the weather will be like tomorrow ',
 'sounds good mate let me know if you need a hand in anything good luck ',
 'i left the car unlocked for a while at the parking lot of the shopping mall ',
 'i saw your son coming out of a theater in the daytime ']

There were additionally sentences with numbers so I changed it manually

In [ ]:
for i, text in enumerate(new_texts):
    if '11' in text or '3' in text or '9' in text:
      print('index : ', i)
      print(text)

index :  291
just finished watching district 9 it was an excellent commentary 
index :  392
do you ever listen to mp3 files on your computer 
index :  2006
i wish i could sleep at 11 that would be amazing 


In [13]:
new_texts[2006] = new_texts[2006].replace('11', 'eleven')
print(new_texts[2006])

new_texts[392] = new_texts[392].replace('mp3', 'm p three')
print(new_texts[392])

new_texts[291] = new_texts[291].replace('9', 'nine')
print(new_texts[291])

i wish i could sleep at eleven that would be amazing 
do you ever listen to m p three files on your computer 
just finished watching district nine it was an excellent commentary 


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here. 
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [14]:
vocab = []
for i in range(len(new_texts)):
    all_text = " ".join(new_texts[i])
    vocab.extend(list(set(all_text)))
    vocab = list(set(vocab))

print(vocab)

['x', 'a', 'g', 'r', 't', 'p', "'", 'z', 'e', 'y', 'd', 'i', 'c', 'm', 'u', 'w', 'v', 'j', ' ', 'h', 's', 'n', 'o', 'k', 'l', 'b', 'q', 'f']


In [15]:
vocab_dict_2 = {v: k for k, v in enumerate(vocab)}
print(vocab_dict_2)

{'x': 0, 'a': 1, 'g': 2, 'r': 3, 't': 4, 'p': 5, "'": 6, 'z': 7, 'e': 8, 'y': 9, 'd': 10, 'i': 11, 'c': 12, 'm': 13, 'u': 14, 'w': 15, 'v': 16, 'j': 17, ' ': 18, 'h': 19, 's': 20, 'n': 21, 'o': 22, 'k': 23, 'l': 24, 'b': 25, 'q': 26, 'f': 27}


Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because: 

- The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- In English, we need to keep the `'` character to differentiate between words, *e.g.*, `"it's"` and `"its"` which have very different meanings.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Timit's training set. 

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [16]:
vocab_dict_2["|"] = vocab_dict_2[" "]
del vocab_dict_2[" "]

In [17]:
vocab_dict_2["[UNK]"] = len(vocab_dict_2)
vocab_dict_2["[PAD]"] = len(vocab_dict_2)
print(len(vocab_dict_2))
print(vocab_dict_2)

30
{'x': 0, 'a': 1, 'g': 2, 'r': 3, 't': 4, 'p': 5, "'": 6, 'z': 7, 'e': 8, 'y': 9, 'd': 10, 'i': 11, 'c': 12, 'm': 13, 'u': 14, 'w': 15, 'v': 16, 'j': 17, 'h': 19, 's': 20, 'n': 21, 'o': 22, 'k': 23, 'l': 24, 'b': 25, 'q': 26, 'f': 27, '|': 18, '[UNK]': 28, '[PAD]': 29}


Cool, now our vocabulary is complete and consists of 30 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 30.

Let's now save the vocabulary as a json file.

In [18]:
import json
with open('/content/drive/MyDrive/korean_english/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict_2, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [19]:
from transformers import Wav2Vec2CTCTokenizer # Might have a error at first run, please try again

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/korean_english/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus, 
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

Wav2Vec2 was pretrained on the audio data of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) and LibriVox which both were sampling with 16kHz. Our fine-tuning dataset, [Timit](hhtps://huggingface.co/datasets/timit_asr), was luckily also sampled with 16kHz. If the fine-tuning dataset would have been sampled with a rate lower or higher than 16kHz, we first would have had to up or downsample the speech signal to match the sampling rate of the data used for pretraining. 



A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. This is **not** recommended for other speech models. For more information, one can take a look at [this](https://github.com/pytorch/fairseq/issues/3227) issue. **Important** If you want to use this notebook to fine-tune [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60), this parameter should be set to `True`.

In [20]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

Great, Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [21]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

If one wants to re-use the just created processor and the fine-tuned model of this notebook, one can mount his/her google drive to the notebook and save all relevant files there. To do so, please uncomment the following lines. 

We will give the fine-tuned model the name `"wav2vec2-base-timit-demo"`.

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo")

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just kept the path to its file in the dataset. `Wav2Vec2` expects the audio file in the format of a 1-dimensional array, so in the first step, let's load all audio files into the dataset object.

Let's first check the serialization format of the downloaded audio files by looking at the first training sample.

Alright, the audio file is saved in the `.pcm` format, so we are gonna transform its format into `.wav`. There are a couple of python-based libraries to read and process audio files, such as [librosa](https://github.com/librosa/librosa), [soundfile](https://github.com/bastibe/python-soundfile), and [audioread](https://github.com/beetbox/audioread). 

`librosa` seems to be the most active and prominent library, but since it depends on `soundfile` for loading of audio files, we will just use `soundfile` directly in this notebook.

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a `map(...)` function accordingly.

In [ ]:
# use .txt filenames to upload .pcm files
print(filename[:3])

['/content/drive/MyDrive/korean_english/files/GSF03031LJH0/SW201105ETRNGSF03031LJH0074.txt', '/content/drive/MyDrive/korean_english/files/GSF03031LJH0/SW201105ETRNGSF03031LJH0091.txt', '/content/drive/MyDrive/korean_english/files/GSF03031LJH0/SW201105ETRNGSF03031LJH0005.txt']


In [ ]:
import soundfile as sf
import wave
import os

channels=1
bits=16
sample_rate=16000 # this dataset was recorded at 16k sample rate

# convert .pcm to .wav

filename_wav = []
for i, name in enumerate(filename):
    if i % 100 == 0:
        print(i)
    pcm_file = name.replace('.txt','.pcm')
    save_file = name.replace('.txt','.wav')
    filename_wav.append(save_file)

    pcmf = open(pcm_file, 'rb')
    pcmdata = pcmf.read()
    pcmf.close()

    wavfile = wave.open(save_file, 'wb')

    wavfile.setnchannels(channels)
    wavfile.setsampwidth(bits // 8)
    wavfile.setframerate(sample_rate)

    wavfile.writeframes(pcmdata)
    wavfile.close()
    #speech_array, sampling_rate = sf.read(name.replace('.txt','.pcm'))


In [ ]:
import IPython.display as ipd
i = 4500
print(new_texts[i])
ipd.Audio(data=filenames[i][:-4] + '.wav', autoplay=True, rate=16000)

let me see your airline ticket and passport please 


There is interval with silence at both start and end parts. We will trim those parts with `librosa.effects.trim` 

In [ ]:
import librosa
import soundfile as sf

# Trim and save

filename_wav_trim = []
for i, name in enumerate(filename_wav):
    if i % 100 == 0:
        print(i)
    y, sr = librosa.load(name,sr=sample_rate)
    yt, index = librosa.effects.trim(y)
    save_file = name[:-4] + '_trim.wav'
    filename_wav_trim.append(save_file)
    sf.write(save_file, yt, sr)

In [ ]:
# Read converted and trimmed .WAV file

speech_arrays = []
sampling_rates = []
for i, name in enumerate(filenames):
    if i % 100 == 0:
        print(i)
    speech_array, sampling_rate = sf.read( name[:-4] + '_trim.wav')
    speech_arrays.append(speech_array)
    sampling_rates.append(sampling_rate)

In [ ]:
# Dump processed speech arrays and sampling rates
import pickle

with open('/content/drive/MyDrive/korean_english/speech_arrays', 'wb') as fp:
    pickle.dump(speech_arrays, fp)
with open('/content/drive/MyDrive/korean_english/sampling_rates', 'wb') as fp:
    pickle.dump(sampling_rates, fp)

In [23]:
# Read processed speech arrays and sampling rate
import pickle

with open ('/content/drive/MyDrive/korean_english/speech_arrays', 'rb') as fp:
    speech_arrays = pickle.load(fp)
with open ('/content/drive/MyDrive/korean_english/sampling_rates', 'rb') as fp:
    sampling_rates = pickle.load(fp)

In [ ]:
# Checking the alignment of the data

assert len(speech_arrays) == len(sampling_rates) == len(new_texts), "each sample's datums are not aligned"

In [ ]:
import IPython.display as ipd

# Check random file and its text 
i = 4500
print(new_texts[i])
print(sampling_rates[i])
ipd.Audio(data=filenames[i][:-4] + '_trim.wav', autoplay=True, rate=22050)

let me see your airline ticket and passport please 
16000


It can be heard, that the speakers change along with their speaking rate, accent, etc. Overall, the recordings sound relatively clear though, which is to be expected from a read speech corpus.

Let's do a final check that the data is correctly prepared, but printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

### Training

Finally, we can process the dataset to the format expected by the model for training. We will again make use of the `map(...)` function.

First, we check that all data samples have the same sampling rate (of 16kHz).
Second, we extract the `input_values` from the loaded audio file. In our case, this includes only normalization, but for other speech models, this step could correspond to extracting, *e.g.* [Log-Mel features](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum). 
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

First we will divide data into train set and test set

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split

speech_arrays_train, speech_arrays_test, new_texts_train, new_texts_test = train_test_split(speech_arrays, new_texts, test_size=0.15, random_state=42)

In [ ]:
input_values_train = processor(speech_arrays_train,sampling_rate=sampling_rates[0]).input_values
input_values_test = processor(speech_arrays_test,sampling_rate=sampling_rates[0]).input_values

with processor.as_target_processor():
   target_train = processor(new_texts_train).input_ids
   target_test = processor(new_texts_test).input_ids

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
type(input_values_train)

numpy.ndarray

Also, this huggingface transformer Wav2vec Trainer requires Datasetclass format so we will transform this ndarray to dataset object format.  
(https://huggingface.co/transformers/custom_datasets.html)

In [ ]:
import torch

class CustomerDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        #item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item = {}
        item['input_values'] = torch.tensor(self.encodings[idx])
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomerDataset(input_values_train, target_train)
test_dataset = CustomerDataset(input_values_test, target_test)

## Training & Evaluation

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [34]:
from datasets import load_dataset, load_metric

wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

In [28]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the [paper](https://arxiv.org/abs/2006.11477) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo", # customer local directory
  #output_dir="./wav2vec2-base-timit-demo",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=25, #30
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 90 and 180 minutes depending on the GPU allocated to this notebook. While the trained model yields satisfying results on *Timit*'s test data, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how Wav2Vec2's [base](https://huggingface.co/facebook/wav2vec2-base), [large](https://huggingface.co/facebook/wav2vec2-large), and [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60) checkpoints can be fine-tuned on any English dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
500,6.995000,2.344070,0.981409,48.970600,14.621000


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
500,6.995000,2.344070,0.981409,48.970600,14.621000
1000,0.844100,0.452271,0.293140,49.715300,14.402000
1500,0.223800,0.392107,0.266038,50.029200,14.312000
2000,0.154000,0.449885,0.257004,50.135300,14.281000
2500,0.100900,0.447991,0.247840,49.909800,14.346000
3000,0.079900,0.480566,0.240508,50.138500,14.280000


TrainOutput(global_step=3175, training_loss=1.3265151316725363, metrics={'train_runtime': 5865.3915, 'train_samples_per_second': 0.541, 'total_flos': 4.256001116504064e+18, 'epoch': 25.0, 'init_mem_cpu_alloc_delta': 52779, 'init_mem_gpu_alloc_delta': 378771968, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 726441, 'train_mem_gpu_alloc_delta': 1114333696, 'train_mem_cpu_peaked_delta': 190301950, 'train_mem_gpu_peaked_delta': 6145681920})

The final WER should be below 0.3 which is reasonable given that state-of-the-art phoneme error rates (PER) are just below 0.1 (see [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit)) and that WER is usually worse than PER.

The resulting model of this notebook has been saved to [patrickvonplaten/wav2vec2-base-timit-demo](https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo).

### Evaluate

In the final part, we evaluate our fine-tuned model on the test set and play around with it a bit.

Let's load the `processor` and `model`.

You can upload fine-tuned processor and model from below information  
1. processor : base + finetuning(Korean-English dataset) ('/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo')
2. processor2 : base + finetuning(huggingface timit_asr dataset) ('/patrickvonplaten/wav2vec2-base-timit-demo')
3. processor3 : base + finetuning(TIMIT) + finetuning_2(korean-English) ('/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo_2)
4. processor4 : Base + finetuning(960 hours of Librispeech) ("/facebook/wav2vec2-base-960h")

**Above personal google drive links will be updated to hugging face model link soon**

In [26]:
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo")
#processor2 = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")
#processor3 = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo_2")
#processor4 = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [29]:
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo/checkpoint-3000")
#model2 = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")
#model3 = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/korean_english/files/wav2vec2-base-timit-demo_2/checkpoint-3000")
#model4 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`. 

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [30]:
# You should match model and processor number
# preprocess test set input

model.to("cuda")

input_values = []

for i, array in enumerate(speech_arrays_test):
    input_value = processor(
        array, 
        sampling_rate=16000, 
        return_tensors="pt"
    ).input_values.to("cuda")

    input_values.append(input_value)

In [32]:
# prediction
import torch

results = []

for i, value in enumerate(input_values):
    with torch.no_grad():
      logits = model(value).logits
    pred_ids = torch.argmax(logits, dim=-1)
    pred_str = processor.batch_decode(pred_ids)[0]
    results.append(pred_str)

Let's compute the overall WER now.

In [ ]:
from datasets import load_dataset, load_metric

wer_metric = load_metric("wer")

In [35]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results, references=new_texts_test)))

Test WER: 0.160


Let's check out how Korean English speech recognition works well without any language models

In [56]:
# Check the result
import pandas as pd
from IPython.display import display, HTML
import random

indexes = random.sample(range(len(results)),10)

df = pd.DataFrame({'predictions' : results, 'target text': new_texts_test})
display(HTML(df.sample(n=10).to_html()))

,predictions,target text
471,how would you feel about going out tonight,how would you feel about going out tonight
458,will you tell me when the toyil is vakent,will you tell me when the toilet is vacant
370,how long does it take from the airport by taxi,how long does it take from the airport by taxi
507,can i buy your lons to make up for being late,can i buy you lunch to make up for being late
299,well i usually listene to music and watchtively at you,well i usually listen to music and watch tv and you
257,one a thirty and one elemental student on janeuarly nineth,one adult and one elementary student on january ninth
156,one way ticket to los angeles and how will you be paing for thi sir,one way ticket to los angeles and how will you be paying for this sir
293,it's easy to find the left at the first light its next whighthotel,it's easy to find turn left at the first light it's next to the white hotel
577,would like to know which one is faster,would like to know which one is faster
558,i'm looking for twety for a six year old boy,i'm looking for a toy for a six year old boy


It becomes clear that the predicted transcriptions are acoustically very similar to the target transcriptions, but often contain spelling or grammatical errors. This shouldn't be very surprising though given that we purely rely on Wav2Vec2 without making use of a language model.